# Data Loader

This is the rebuild of data process functions. 

## Load Packages

In [ ]:
import os
import numpy as np
import collections

## Tool Functions

### Label Function

Converge TCP attack type to int. 

In [ ]:
def label(flag: str) -> int:
    return {
        '-': 0,
        'snmpgetattack': 1,
        'warez': 2,
        'portsweep': 3,
        'nmap': 4,
        'warezclient': 5,
        'ipsweep': 6,
        'dict': 7,
        'neptune': 8,
        'smurf': 9,
        'pod': 10,
        'snmpguess': 11,
        'teardrop': 12,
        'satan': 13,
        'httptunnel-e': 14,
        'ignore': 15,
        'mscan': 16,
        'guest': 17,
        'rootkit': 18,
        'back': 19,
        'apache2': 20,
        'processtable': 21,
        'mailbomb': 22,
        'smurfttl': 23,
        'saint': 24
    }.get(flag)

In [ ]:
def line_split(line: str) -> tuple:
    src_ip, dst_ip, port, time, connect_type = line.split('\t')
    return int(src_ip), int(dst_ip), int(port), int(time), int(connect_type)

## Parameters

In [ ]:
# Data File Path

train_set_path = 'data/train'
valid_set_path = 'data/valid_query'
test_set_path = 'data/test_query'

# Features

src_in_degree = collections.OrderedDict()
src_out_degree = collections.OrderedDict()
dst_in_degree = collections.OrderedDict()
dst_out_degree = collections.OrderedDict()

port_num = collections.OrderedDict()
port_list = collections.OrderedDict()
time_len = collections.OrderedDict()
connect_num = collections.OrderedDict()

# Tool Parameters

nodes_origin = []
edges_origin = {}

node_in_degree = {}
node_out_degree = {}

edge_src_list = {}
edge_dst_list = {}

# Count Parameters

edge_index = 0

## Load Data

### Singel File

In [ ]:
path = 'data/train/train_000.txt'
data = open(path)

for line in data:
    src_ip, dst_ip, port, time, connect_type = line_split(line)
    
    # Repeat Connections
    if (src_ip, dst_ip) in edges_origin.keys():
        edge_index_temp = edges_origin[(src_ip, dst_ip)]
        
        # Assign Features
        time_len[edge_index_temp] += time
        connect_num[edge_index_temp] += 1
        if port in port_list[edge_index_temp]:
            pass
        else:
            port_list[edge_index_temp].append(port)
    
    # New Connections     
    else:
        # Record Origin Node and Edge
        nodes_origin.append(src_ip)
        nodes_origin.append(dst_ip)
        edges_origin[(src_ip, dst_ip)] = edge_index
        
        # Update Node Parameters
        if src_ip in node_out_degree.keys():
            node_out_degree[src_ip] += 1
        else:
            node_out_degree[src_ip] = 1
        if dst_ip in node_in_degree.keys():
            node_in_degree[dst_ip] += 1
        else:
            node_in_degree[dst_ip] = 1
        
        # Update Edge Parameters
        if edge_index in edge_src_list.keys():
            edge_src_list[edge_index].append(src_ip)
        else:
            edge_src_list[edge_index] = [src_ip]
        if edge_index in edge_dst_list.keys():
            edge_dst_list[edge_index].append(dst_ip)
        else:
            edge_dst_list[edge_index] = [dst_ip]
            
        # Assign Features
        port_list[edge_index] = [port]
        time_len[edge_index] = time
        connect_num[edge_index] = 1

In [ ]:
nodes_origin = list(set(nodes_origin))

for i in range(len(edge_src_list.keys())):
    edge_src_list[i]